In [1]:
import spotipy
import spotipy.util as util
import pandas as pd
import re


In [202]:
concerts = pd.read_csv("concerts_clean.csv", index_col=0)
concerts['date'] = pd.to_datetime(concerts['date'], format = "%Y/%m/%d")

In [204]:
concerts.head()

,date,url,venue,artist,location,time
0,2017-02-02,https://www.songkick.com/metro_areas/7644-us-n...,Knitting Factory,Don Giovanni Records Showcase 2017,new-york,NaN
1,2017-02-03,https://www.songkick.com/metro_areas/7644-us-n...,Madison Square Garden,The Lumineers,new-york,NaN
2,2017-02-03,https://www.songkick.com/metro_areas/7644-us-n...,Terminal 5,AFI,new-york,19:00:00-0500
3,2017-02-03,https://www.songkick.com/metro_areas/7644-us-n...,Webster Hall,Keys N Krates (DJ set),new-york,22:00:00-0500
4,2017-02-03,https://www.songkick.com/metro_areas/7644-us-n...,Music Hall of Williamsburg,Bear's Den,new-york,20:00:00-0500


In [205]:
cityview = concerts.groupby('location')


In [ ]:
cityview.describe()[0:10]

In [211]:
venueview = concerts.groupby('venue')

In [281]:
def get_venue_artists(venue, start_date = "2017-01-01", end_date = "2017-12-31"):
    concert_dates = pd.date_range(start = start_date, end = end_date, freq = 'D')
    artists = []
    for show in concerts.loc[concerts['venue'] == venue, ['date', 'artist']].itertuples():
        if show[1] in concert_dates:
            artists.append(show)
    return artists

In [283]:
#NOTE: I would like to add a way to validate the artist returned by the search here. To do later...
def get_artist_ids(artists):
    sp = spotipy.Spotify()
    artist_plus_ids = []
    for artist in artists:
        search = sp.search(q=artist, type = 'artist', limit = 1)
        try:
            artist_plus_ids.append((search['artists']['items'][0]['name'] ,  search['artists']['items'][0]['id']))
        except IndexError:
            pass
    return artist_plus_ids

In [284]:
def get_venue_artist_ids(venue, start_date = "2017-01-01", end_date = "2017-12-31"):
    sp = spotipy.Spotify()
    artists = get_venue_artists(venue, start_date, end_date)
    artist_plus_ids = []
    for artist in artists:
        search = sp.search(q=artist[2], type = 'artist', limit = 1)
        try:
            artist_plus_ids.append((search['artists']['items'][0]['name'] ,  search['artists']['items'][0]['id']))
        except IndexError:
            pass
    return artist_plus_ids


In [286]:
def create_venue_songlist(venue, start_date = "2017-01-01", end_date = "2017-12-31"):
    sp = spotipy.Spotify()
    songlist = []
    for artist in get_venue_artist_ids(venue, start_date, end_date):
        artist_tracks = sp.artist_top_tracks(artist[1])['tracks']
        if len(artist_tracks) >= 5:
            for track in range(0, 5):
                songlist.append((artist_tracks[track]['name'], \
                                 artist_tracks[track]['id']))
    return songlist
    

In [169]:
def prepare_song_id_list(songlist):
    song_ids = []
    for song in songlist:
        song_ids.append(str(song[1]))
    return song_ids

In [313]:
def create_venue_songlist_ids(venue, start_date = "2017-01-01", end_date = "2017-12-31"):
    songlist = create_venue_songlist(venue, start_date, end_date)
    print type(songlist)
    if len(songlist) >= 100:
        songlist = songlist[0:99]
    ptitle = ("%s from %s to %s") % (venue, start_date, end_date)
    return (ptitle, prepare_song_id_list(songlist))

In [314]:
####Now that we have a list of song IDs, let's authenticate a spotify login, and create the playlist

In [315]:
SPOTIPY_CLIENT_ID = '1723f7534a1c4cde9efe16b0b2753d7f'
SPOTIPY_CLIENT_SECRET = 'e3e4b35b367a430c8b2900ef5d6165a7'
SPOTIPY_REDIRECT_URI = 'http://localhost:8888'

In [329]:
token = util.prompt_for_user_token("", 'playlist-modify-public', SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET, SPOTIPY_REDIRECT_URI)

In [365]:
import random
def random_venue_tester():
    x = random.choice(range(0, len(concerts['venue'])))
    return concerts['venue'][x]

In [394]:
random_venue_tester()

'The Regent Theater'

In [398]:
username = 'afeierman'
playlist_prepped = create_venue_songlist_ids(random_venue_tester(), '2017-02-01', '2017-02-15')
playlist_title = playlist_prepped[0]
track_ids = playlist_prepped[1]

<type 'list'>


In [399]:
if token:
    sp = spotipy.Spotify(auth=token)
    sp.trace = False #Not sure if this is needed
    sp.user_playlist_create(username, playlist_title)
    playlists = sp.user_playlists(username)
    for playlist in playlists['items']:
        if playlist['name'] == playlist_title:
            playlist_id = playlist['id']
            break
    results = sp.user_playlist_add_tracks(username, playlist_id, track_ids)
    print results
else:
    print "Can't get token"

{u'snapshot_id': u'AQK9J0Em9r15UAlO2zzMNBXfXT0QsNxtwctROxImUUhGxidpOsvRaWpTbu33aWu7'}
